In [1]:
# canonical form identifying user LLM question
# "define user ask llm"
"what is the llama 2 model?"
"tell me about Meta's new LLM"
"what are the differences between Falcon and Llama?"

'what are the differences between Falcon and Llama?'

In [2]:
# building the knowledge base using Llama 2 ArXiv papers 
# stored in HuggingFace. Download
from datasets import load_dataset

data = load_dataset(
    "jamescalam/llama-2-arxiv-papers-chunked",
    split="train")
print("Data:")
print(data)
print("\n")
    

Data:
Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})




In [3]:
data[0]

{'doi': '1102.0183',
 'chunk-id': '0',
 'chunk': 'High-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nTechnical Report No. IDSIA-01-11\nJanuary 2011\nIDSIA / USI-SUPSI\nDalle Molle Institute for Arti\x0ccial Intelligence\nGalleria 2, 6928 Manno, Switzerland\nIDSIA is a joint institute of both University of Lugano (USI) and University of Applied Sciences of Southern Switzerland (SUPSI),\nand was founded in 1988 by the Dalle Molle Foundation which promoted quality of life.\nThis work was partially supported by the Swiss Commission for Technology and Innovation (CTI), Project n. 9688.1 IFF:\nIntelligent Fill in Form.arXiv:1102.0183v1  [cs.AI]  1 Feb 2011\nTechnical Report No. IDSIA-01-11 1\nHigh-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nJanuary 2011\nAbs

In [4]:
# reformat the data for a couple of columns
data = data.map(lambda x: {
    'uid': f"{x['doi']}--{x['chunk-id']}"
})
data

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references', 'uid'],
    num_rows: 4838
})

In [5]:
# move the data to pandas
data = data.to_pandas()

# drop the irrelevant fields
data = data[['uid', 'chunk', 'title', 'source']]
print(data)

                  uid                                              chunk  \
0        1102.0183--0  High-Performance Neural Networks\nfor Visual O...   
1        1102.0183--1  January 2011\nAbstract\nWe present a fast, ful...   
2        1102.0183--2  promising architectures for such tasks. The mo...   
3        1102.0183--3  Mutch and Lowe, 2008), whose lters are xed, ...   
4        1102.0183--4  We evaluate various networks on the handwritte...   
...               ...                                                ...   
4833  2307.09288--315  BytheCentralLimitTheorem, Zntendstowardsastand...   
4834  2307.09288--316  Table 52 presents a model card (Mitchell et al...   
4835  2307.09288--317  models will be released as we improve model sa...   
4836  2307.09288--318  Training Factors We usedcustomtraininglibrarie...   
4837  2307.09288--319  Evaluation Results\nSee evaluations for pretra...   

                                                  title  \
0     High-Performance Neura

In [6]:
# get the environment variables and create embeddings
from dotenv import dotenv_values
from openai import OpenAI

model_id = "text-embedding-ada-002"

# get the api keys needed
config = dotenv_values(".env")
openai_api_key = config["OPENAI_API_KEY"]
print(openai_api_key)

client = OpenAI(
    api_key=openai_api_key
)
embedding = client.embeddings.create(
    input=["We would have some text to embed here",
        "And another chunk here"],
    model=model_id
)
print("embedding data:")
#embedding.data
print(embedding)

#dict_keys(['object', 'data', 'model', 'usage'])


sk-proj-3YRWRWnvcDDLVT9Ob3yQT3BlbkFJFBjjYn2yJfNoOZ1pFL0f
embedding data:
CreateEmbeddingResponse(data=[Embedding(embedding=[-0.018025800585746765, 0.01235796045511961, 0.003275506431236863, -0.006072222720831633, -0.0005301468190737069, 0.02164907567203045, -0.00848234910517931, -0.006974807009100914, -0.010352217592298985, -0.009006429463624954, 0.0016482672654092312, 0.024263011291623116, 0.009252294898033142, -0.0055190264247357845, 0.0017016457859426737, 0.018750455230474472, 0.01530834287405014, 0.005431679543107748, 0.0003410975041333586, -0.035094019025564194, -0.008372356183826923, 0.001498645287938416, -0.004098831210285425, 0.004186178091913462, -0.02551821805536747, 0.000981034361757338, 0.02230902947485447, -0.02040681056678295, -0.0109604112803936, -0.019966840744018555, 0.024547696113586426, -0.017171742394566536, -0.008902907371520996, -0.02221844717860222, -0.024677099660038948, -0.006547777913510799, -0.006441020872443914, 0.0022613126784563065, 0.026760483160614967, -

In [7]:
embed_data = embedding.data
e1_len = len(embed_data[0].embedding)
e2_len = len(embed_data[1].embedding)
print(f"e1 len = {e1_len}")
print(f"e2 len = {e2_len}")

e1 len = 1536
e2 len = 1536


In [8]:
# create vector index for PineCone serverless
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

# get the list of indices
index_name = "nemo-guardrails-rag"
pc = PineconeGRPC()
indexes = pc.list_indexes().indexes
names = [x['name'] for x in indexes]

# check if index Exists -> create if DNE
if index_name not in names:
    pc.create_index(
        name=index_name,
        dimension=e1_len,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-west-2'))

In [9]:
#connect to the index
index = pc.Index(index_name)

# ensure the index and data come from Dataset/Pandas
print(index)
print(data)

                  uid                                              chunk  \
0        1102.0183--0  High-Performance Neural Networks\nfor Visual O...   
1        1102.0183--1  January 2011\nAbstract\nWe present a fast, ful...   
2        1102.0183--2  promising architectures for such tasks. The mo...   
3        1102.0183--3  Mutch and Lowe, 2008), whose lters are xed, ...   
4        1102.0183--4  We evaluate various networks on the handwritte...   
...               ...                                                ...   
4833  2307.09288--315  BytheCentralLimitTheorem, Zntendstowardsastand...   
4834  2307.09288--316  Table 52 presents a model card (Mitchell et al...   
4835  2307.09288--317  models will be released as we improve model sa...   
4836  2307.09288--318  Training Factors We usedcustomtraininglibrarie...   
4837  2307.09288--319  Evaluation Results\nSee evaluations for pretra...   

                                                  title  \
0     High-Performance Neura

In [82]:
# upload vector data and metadata to PineCone
from tqdm.auto import tqdm

# how many embeddings we create/insert
batch_size = 100

# use tqdm to track uploaded batches
for i in tqdm(range(0, len(data), batch_size)):
    # find end of batch
    idx_end = min(len(data), i+batch_size)
    batch = data[i:idx_end]

    # get ids
    ids_batch = batch['uid'].to_list()

    # get texts to encode
    texts = batch['chunk'].to_list()

    # create embeddings from txt
    res = client.embeddings.create(
        input=texts,
        model=model_id)
    embeds = [record.embedding for record in res.data]

    # create metadata
    metadata = [{
        'chunk': x['chunk'],
        'source': x['source']
    } for _, x in batch.iterrows()]

    # upsert data
    upsert_data = list(zip(ids_batch, embeds, metadata))
    index.upsert(vectors=upsert_data)


  0%|          | 0/49 [00:00<?, ?it/s]

In [51]:
# retrieve data from PC index using embedding
async def retrieve(query: str) -> list:
    # create query embedding
    query_embedding = client.embeddings.create(
        input=[query],
        model=model_id)
    query_data = query_embedding.data
    vector_query = query_data[0].embedding

    # get relevant contexts from PC
    res = index.query(vector_query, top_k=5, include_metadata=True)

    # get list of retrieved texts
    contexts = [x['metadata']['chunk'] for x in res['matches']]
    return contexts

# to run an aysnc() method we need to call "await" prior
# await retrieve(query="tell me about llama 3")

In [52]:
# create a rag func where the contexts comes from PC
# and query is a user related query
async def rag(query: str, contexts: list) -> str:
    print("> RAG activated")
    context_str = "\n".join(contexts)
    print("Context str:")
    print(context_str)
    print("\n")

    # place query and contexts into RAG prompt
    prompt = f"""You are a helpful assistant, below is a query from a user and
    some relevant contexts. Answer the question given the information in those
    contexts. If you cannot find the answer to the question, say "I don't know".

    Contexts: {context_str}

    Query: {query}

    Answer: """

    # generate the answer
    res = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100)
    
    return res.choices[0].text

In [57]:
# implement guardrails for config initialization
yaml_content = """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo-instruct
"""

rag_colang_content = """
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a personal assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help

# define RAG intents and flow
define user ask llms
    "tell me about llama 2?"
    "what is large language model"
    "where did meta's new model come from?"
    "what is the falcon model?"
    "have you ever meta llama?"

define flow llms
    user ask llms
    $contexts = execute retrieve(query=$last_user_message)
    $answer = execute rag(query=$last_user_message, contexts=$contexts)
    bot $answer
"""

In [58]:
from nemoguardrails import LLMRails, RailsConfig
import os

os.environ["OPENAI_API_KEY"] = openai_api_key
# init rails config
config = RailsConfig.from_content(
    colang_content=rag_colang_content,
    yaml_content=yaml_content
)

# create the rails 
rag_rails = LLMRails(config)

In [59]:
# register the actions in rag rails
rag_rails.register_action(action=retrieve, name="retrieve")
rag_rails.register_action(action=rag, name="rag")

In [28]:
# generate questions/answers
await rag_rails.generate_async(prompt="hello")

'Hello again! Is there something specific you would like me to assist you with?'

In [29]:
await rag_rails.generate_async(prompt="what is your party affiliation?")

"I'm a personal assistant, I don't like to talk of politics.\nIs there anything else I can assist you with?"

In [60]:
await rag_rails.generate_async(prompt="tell me about the origins of llms")

> RAG activated
Context str:
complex reasoning tasks requiring expert knowledge across a wide range of ﬁelds, including in specialized
domains such as programming and creative writing. They enable interaction with humans through intuitive
chat interfaces, which has led to rapid and widespread adoption among the general public.
ThecapabilitiesofLLMsareremarkableconsideringtheseeminglystraightforwardnatureofthetraining
methodology. Auto-regressivetransformersarepretrainedonanextensivecorpusofself-superviseddata,
followed by alignment with human preferences via techniques such as Reinforcement Learning with Human
Feedback(RLHF).Althoughthetrainingmethodologyissimple,highcomputationalrequirementshave
limited the development of LLMs to a few players. There have been public releases of pretrained LLMs
(such as BLOOM (Scao et al., 2022), LLaMa-1 (Touvron et al., 2023), and Falcon (Penedo et al., 2023)) that
match the performance of closed pretrained competitors like GPT-3 (Brown et al., 2020)

'LLMs were first developed as a way to handle complex reasoning tasks requiring expert knowledge across a wide range of fields. They have since been widely adopted by the general public due to their intuitive chat interfaces and impressive capabilities. The training methodology for LLMs involves pretraining on a large corpus of self-supervised data and then aligning with human preferences through techniques like Reinforcement Learning with Human Feedback. However, the development of LLMs has been limited due to high computational requirements. Recently, there have'

In [50]:
await rag_rails.generate_async(prompt="tell me about llama 2")

> RAG activated
Context str:
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
ourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety
asChatGPT,BARD,andClaude. TheseclosedproductLLMsareheavilyﬁne-tunedtoalignwithhuman
preferences, which greatly e

'\nLlama 2 is a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters. These models, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc, are optimized for dialogue use cases and have been shown to outperform open-source chat models on most benchmarks. They are also on par with some closed-source models in terms of human evaluations for helpfulness and'

In [61]:
# rag vs no rag implementation
no_rag_colang_content = """
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a personal assistant, I don't like to talk of politics."

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

# init rails config
config = RailsConfig.from_content(
    colang_content=no_rag_colang_content,
    yaml_content=yaml_content
)
# create the rails obj
no_rag_rails = LLMRails(config)

In [63]:
await no_rag_rails.generate_async(prompt="tell me about llama 2")


"I'm sorry, I don't know about llama 2. Can I help you with anything else?"

In [64]:
# no RAG
await no_rag_rails.generate_async(
    prompt="what was red teaming used for in llama 2 training?"
)

'Red teaming was used in the Llama 2 training to simulate real-world scenarios and identify any vulnerabilities or weaknesses in the training program. This helps to improve the overall effectiveness and preparedness of the training.'

In [65]:
# with RAG
await rag_rails.generate_async(
    prompt="what was red teaming used for in llama 2 training?"
)

> RAG activated
Context str:
vague answers due to context distillation). We thus leverage the safety reward model to decide whether to
use safety context distillation – we keep the context-distilled output only on the examples where it gets a
betterrewardmodelscorethantheoriginalanswer. Wenoticethatthisisparticularlyhelpfulonprompts
that the model is very bad at, but limits the negative impact of context distillation (see Figure 16b).
4.3 Red Teaming
GivenhowbroadthecapabilitiesofLLMsareandhowvariedtheirtrainingdatais,itisinsuﬃcienttoidentify
risks solely via ex post facto usage and analysis. Rather, as has been done for other LLMs, we performed
various kinds of proactive risk identiﬁcation, colloquially called “red teaming,“ based on the term commonly
used within computer security. This kind of granular analysis is very important because safety is a long-tail
issue,inwhichevenveryinfrequentedgecasescancausenoticeableproblems. Evenifquantitativescores
report good results, these types o

' Red teaming was used in Llama 2 training for proactive risk identification and to measure the robustness of each new model as it was released internally. It involved conducting various exercises with internal employees, contract workers, and external vendors to identify potential risks and improve the safety and effectiveness of the models.'